<a href="https://colab.research.google.com/github/guikingoak/Estudos_Big_Data_Analytics/blob/main/ExCarros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

Exercício 1: Carregar e Unir os Datasets
Enunciado: Carregue os datasets `cars1.csv` e `cars2.csv` como DataFrames no PySpark. Em seguida, una os dois DataFrames utilizando `UNION` para criar um único DataFrame que contenha todos os dados.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, avg

spark = SparkSession.builder \
    .appName("Exercicio Spark Merge e Filter") \
    .getOrCreate()

cars1 = spark.read.csv("cars1.csv", header=True, inferSchema=True)
cars2 = spark.read.csv("cars2.csv", header=True, inferSchema=True)

cars1_cols = cars1.columns
cars2_cols = cars2.columns

missing_cols_cars1 = [col_name for col_name in cars2_cols if col_name not in cars1_cols]
missing_cols_cars2 = [col_name for col_name in cars1_cols if col_name not in cars2_cols]


for col_name in missing_cols_cars1:
    cars1 = cars1.withColumn(col_name, lit(None).cast(cars2.schema[cars2_cols.index(col_name)].dataType))
for col_name in missing_cols_cars2:
    cars2 = cars2.withColumn(col_name, lit(None).cast(cars1.schema[cars1_cols.index(col_name)].dataType))

cars_union = cars1.union(cars2)

cars_union.show()

+----+---------+------------+----------+------+------------+-----+------+--------------------+----+----+----+----+----+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model|origin|                 car| _c9|_c10|_c11|_c12|_c13|
+----+---------+------------+----------+------+------------+-----+------+--------------------+----+----+----+----+----+
|18.0|        8|         307|       130|  3504|        12.0|   70|     1|chevrolet chevell...|NULL|NULL|NULL|NULL|NULL|
|15.0|        8|         350|       165|  3693|        11.5|   70|     1|   buick skylark 320|NULL|NULL|NULL|NULL|NULL|
|18.0|        8|         318|       150|  3436|        11.0|   70|     1|  plymouth satellite|NULL|NULL|NULL|NULL|NULL|
|16.0|        8|         304|       150|  3433|        12.0|   70|     1|       amc rebel sst|NULL|NULL|NULL|NULL|NULL|
|17.0|        8|         302|       140|  3449|        10.5|   70|     1|         ford torino|NULL|NULL|NULL|NULL|NULL|
|15.0|        8|         429|       198|

Exercício 2: Filtrar e Agrupar Dados
Enunciado: A partir do DataFrame unido, filtre os carros que possuem mais de 4 cilindros. Em seguida, agrupe os dados por número de cilindros (`cylinders`) e calcule a média de `mpg` (milhas por galão) para cada grupo.


In [ ]:
from pyspark.sql.functions import col

filtered_cars = cars_union.filter(col("cylinders") > 4)

avg_mpg_by_cylinders = filtered_cars.groupBy("cylinders").agg(avg("mpg").alias("avg_mpg"))

avg_mpg_by_cylinders.show()

+---------+------------------+
|cylinders|           avg_mpg|
+---------+------------------+
|        6|19.985714285714288|
|        8|14.963106796116502|
|        5|27.366666666666664|
+---------+------------------+



Exercício 3: Join entre DataFrames
Enunciado: Considere que você adicionou uma coluna de `ID` única para ambos os DataFrames `cars1` e `cars2`. Realize um `JOIN` entre os dois DataFrames utilizando a coluna `car` (nome do carro). Use um `INNER JOIN` para trazer apenas os registros que estão presentes em ambos os datasets.

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

cars1 = cars1.withColumn("ID", monotonically_increasing_id())
cars2 = cars2.withColumn("ID", monotonically_increasing_id())

joined_df = cars1.join(cars2, on="car", how="inner")

joined_df.show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+----+----+----+----+----+---+----+---------+------------+----------+------+------------+-----+------+----+----+----+----+----+---+
|                 car| mpg|cylinders|displacement|horsepower|weight|acceleration|model|origin| _c9|_c10|_c11|_c12|_c13| ID| mpg|cylinders|displacement|horsepower|weight|acceleration|model|origin| _c9|_c10|_c11|_c12|_c13| ID|
+--------------------+----+---------+------------+----------+------+------------+-----+------+----+----+----+----+----+---+----+---------+------------+----------+------+------------+-----+------+----+----+----+----+----+---+
|buick estate wago...|14.0|        8|         455|       225|  3086|        10.0|   70|     1|NULL|NULL|NULL|NULL|NULL| 13|16.9|        8|         350|       155|  4360|        14.9|   79|     1|NULL|NULL|NULL|NULL|NULL| 91|
|         peugeot 504|25.0|        4|         110|        87|  2672|        17.5|   70|     2|NULL|N

Exercício 4: Modificação dos Dados (Alterar Colunas)
Enunciado: Crie uma nova coluna chamada `performance` que classifique os carros da seguinte forma:
- Se o carro tem mais de 20 `mpg`, classifique como "Alta".
- Se o carro tem entre 15 e 20 `mpg`, classifique como "Média".
- Se o carro tem menos de 15 `mpg`, classifique como "Baixa".

In [ ]:
from pyspark.sql.functions import when

cars_union = cars_union.withColumn("performance", when(cars_union["mpg"] > 20, "Alta")
                                    .when((cars_union["mpg"] >= 15) & (cars_union["mpg"] <= 20), "Média")
                                    .otherwise("Baixa"))

cars_union.show()

+----+---------+------------+----------+------+------------+-----+------+--------------------+----+----+----+----+----+-----------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model|origin|                 car| _c9|_c10|_c11|_c12|_c13|performance|
+----+---------+------------+----------+------+------------+-----+------+--------------------+----+----+----+----+----+-----------+
|18.0|        8|         307|       130|  3504|        12.0|   70|     1|chevrolet chevell...|NULL|NULL|NULL|NULL|NULL|      Média|
|15.0|        8|         350|       165|  3693|        11.5|   70|     1|   buick skylark 320|NULL|NULL|NULL|NULL|NULL|      Média|
|18.0|        8|         318|       150|  3436|        11.0|   70|     1|  plymouth satellite|NULL|NULL|NULL|NULL|NULL|      Média|
|16.0|        8|         304|       150|  3433|        12.0|   70|     1|       amc rebel sst|NULL|NULL|NULL|NULL|NULL|      Média|
|17.0|        8|         302|       140|  3449|        10.5|   70|     1|   

Exercício 5: Contagem de Carros por Origem
Enunciado: Utilize o dataset unido para contar quantos carros há por `origin` (país de origem), e mostre o resultado ordenado de forma decrescente.

In [ ]:
cars_union.groupBy("origin").count().orderBy(col("count").desc()).show()

+------+-----+
|origin|count|
+------+-----+
|     1|  249|
|     3|   79|
|     2|   70|
+------+-----+



Exercício 6: Conversão de Colunas
Enunciado: A coluna `horsepower` contém valores que podem estar no formato string. Converta essa coluna para o tipo inteiro e, em seguida, calcule a média de `horsepower` para todos os carros.

In [ ]:
cars_union =  cars_union.withColumn("horsepower", col("horsepower").cast("int"))
cars_union.select(avg("horsepower")).show()

+------------------+
|   avg(horsepower)|
+------------------+
|104.46938775510205|
+------------------+



Exercício 7: Manipulação de Datas
Enunciado: Suponha que você adicionou uma coluna de data ao DataFrame indicando o ano em que cada carro foi registrado. Filtre o DataFrame para trazer apenas os carros com horse-power maior que 160



In [ ]:
cars_union = cars_union.withColumn("year", col("year").cast("int"))

In [ ]:
cars_union.filter(col("horsepower") > 160).show()

+----+---------+------------+----------+------+------------+-----+------+--------------------+----+----+----+----+----+-----------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model|origin|                 car| _c9|_c10|_c11|_c12|_c13|performance|
+----+---------+------------+----------+------+------------+-----+------+--------------------+----+----+----+----+----+-----------+
|15.0|        8|         350|       165|  3693|        11.5|   70|     1|   buick skylark 320|NULL|NULL|NULL|NULL|NULL|      Média|
|15.0|        8|         429|       198|  4341|        10.0|   70|     1|    ford galaxie 500|NULL|NULL|NULL|NULL|NULL|      Média|
|14.0|        8|         454|       220|  4354|         9.0|   70|     1|    chevrolet impala|NULL|NULL|NULL|NULL|NULL|      Baixa|
|14.0|        8|         440|       215|  4312|         8.5|   70|     1|   plymouth fury iii|NULL|NULL|NULL|NULL|NULL|      Baixa|
|14.0|        8|         455|       225|  4425|        10.0|   70|     1|   